# K-Centers

Just some boilerplate to include all of the packages we need, and set up some configuration:

In [ ]:
import numpy as np
import pandas as pd

from sklearn.utils import check_random_state

import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# Enable high resolution PNGs
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': '#DFDFE5'}
sns.set(context='notebook', style='darkgrid', rc=rc)



### Indexing in numpy

If you're already familiar with numpy indexing, you can skip to the next cell.

Numpy has some great ways to index arrays, but they can sometimes be confusing to the uninitiated. If you're familiar with MATLAB arrays, [this cheat sheet](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html) is pretty useful.

[The documentation on numpy indexing][1] is also fairly clear and has nice, simple examples. We'll be using most of the techniques on that page, like index arrays and boolean indexing.

One thing to note: if you use numpy indexing syntax to retrieve a single row of an array, this will often remove one of the dimensions from the shape of the result. Scikit-learn functions mostly don't like one-dimensional arrays (because they're ambiguous) and will complain. You can use `np.newaxis` to give you the shape you need (note that scikit-learn functions will usually recommend that you use `.reshape(-1,1)`, but this tends to be less intuitive -- examples are included below for comparison).

    > X = np.random.randint(low=0, high=10, size=(2,5))
    > print X
    [[6 5 2 1 5]
     [3 0 8 4 6]]
    
    > x = X[1]
    > print 'The shape of x is', x.shape
    > print x
    The shape of x is (5L,)
    [3 0 8 4 6]

For a column:

    > print 'The shape of x[:,np.newaxis] is', x[:,np.newaxis].shape
    > print x[:,np.newaxis]
    The shape of x[:,np.newaxis] is (5L, 1L)
    [[3]
     [0]
     [8]
     [4]
     [6]]

    > print 'The shape of x.reshape(-1,1) is', x.reshape(-1,1).shape
    > print x.reshape(-1,1)
    The shape of x.reshape(-1,1) is (5L, 1L)
    [[3]
     [0]
     [8]
     [4]
     [6]]

For a row:

    > print 'The shape of x[np.newaxis,:] is', x[np.newaxis,:].shape
    > print x[np.newaxis,:]
    The shape of x[np.newaxis,:] is (1L, 5L)
    [[3 0 8 4 6]]

    > print 'The shape of x.reshape(1,-1) is', x.reshape(1,-1).shape
    > print x.reshape(1,-1)
    The shape of x.reshape(1,-1) is (1L, 5L)
    [[3 0 8 4 6]]

[1]: http://docs.scipy.org/doc/numpy/user/basics.indexing.html

### Broadcasting in Numpy

We'll also be using [broadcasting][2]. The [broadcasting documentation][2] is very simple and explanatory, so we don't need to get very far into it here.

[2]: http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

## Problem 1

Below is a function that computes Euclidean distance from some point in the dataset and every point in the dataset. That is, given $i$, and a matrix $X$ whose rows are points $x_j$, compute
$$ \|x_i - x_j\| $$
for every $0\leq j< \text{n_samples}$.

Recall that 
$$ \|x\| = \sqrt{\sum_{k=0}^{\text{n_features}-1} x_k^2} $$

There will be code that you need to add to make the tests pass.

In [ ]:
def euclid_dist(idx, X):
    """Compute distances between X[idx,:] and every row of X
    Parameters
    ----------
    idx : int
        An index in the range [0,X.shape[0])
    X : array of floats, shape (n_samples, n_features)
        An array where every row represents a point
    Returns
    -------
    distances : float ndarray with shape (n_samples,)
        distances between X[idx,:] and every row of X
    """

    row = X[idx]    
    # The shape is wrong though:
    row = row[np.newaxis,:]

    # Create a new array D with the same shape as X so that a row i
    # of D is X[i] - row
    # HINT: Remember, we can use broadcasting rules
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # We're computing Euclidean distance. Modify D so that every 
    # element of D is now the square of what it was before.
    # YOUR CODE HERE
    raise NotImplementedError()

    # Now sum everything up along the feature dimension:
    D = D.sum(axis=1)
    # "axis" corresponds to the dimension that we want to sum.
    
    # We're computing Euclidean distance. Modify D so that the 
    # proper computation happens after summation.
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return D

In [ ]:
X = np.eye(4)
d = euclid_dist(0,X)
assert(d.shape == (4,))
assert(np.isclose(d, np.array([0,np.sqrt(2),np.sqrt(2),np.sqrt(2)])).all())

d = euclid_dist(3,X)
assert(d.shape == (4,))
assert(np.isclose(d, np.array([np.sqrt(2),np.sqrt(2),np.sqrt(2),0])).all())

## Problem 2

Now we're going to implement the FurthestPoint algorithm from the lecture notes. Some of the work has been done, like setting up arrays to hold data. You'll be in charge of implementing the innards of the loop.

Remember that while you can always loop over the arrays to compute the values you need, numpy has ways to repeat most of the operations internally and far more efficiently. Implementing loops yourself is still allowed, however, and you will receive full credit as long as the tests pass.

In [ ]:


def my_k_centers(X, n_clusters, seed=None):
    """K-centers clustering algorithm.
    Parameters
    ----------
    X : array of floats, shape (n_samples, n_features)
        The observations to cluster.
    n_clusters : int
        The number of clusters to form as well as the number of
        centers to generate.
    seed : int
        The index of the initial point to choose as a cluster center.
    Returns
    -------
    centers : float ndarray with shape (k, n_features)
        Centers found by k-centers.
    center_indices : float ndarray with shape (k, n_features)
        Centers found by k-centers.
    label : integer ndarray with shape (n_samples,)
        label[i] is the code or index of the center the
        i'th observation is closest to.
    clust_size : float
        The final value of the size criterion (max distance to the closest
        centers over all observations in the training set).
    """
    # X has the information we need for numbers of samples and features
    n_samples, n_features = X.shape

    # Set the id of the first cluster center to seed
    cur_id = seed

    # We're going to fill this, so just make it empty
    centers = np.empty((n_clusters, n_features), dtype=X.dtype)
    
    # For recording the indices of the cluster centers
    # i.e., X[center_indices,:] == centers; that is, center_indices will
    # be an inex array.
    center_indices = np.empty((n_clusters,), dtype=np.int32)
    center_indices.fill(-1)

    # Make an array for the distances, and fill it with infinity
    mindist = np.empty(n_samples, dtype=np.float64)
    mindist.fill(np.infty)
    
    # Make an array for the labels (integers) and fill with -1
    labels = np.empty(n_samples, dtype=np.int32)
    labels.fill(-1)
    
    # Now find the farthest point from the 
    for center_id in range(n_clusters):
        # Update center_indices
        # YOUR CODE HERE
        raise NotImplementedError()

        # Compute distances to new center
        dist = euclid_dist(cur_id, X)
        
        # Update the labels of the points that 
        # are closer to the new center
        # HINT: use boolean indexing to change just the elements
        #       that you want to change
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Update mindist with the new minimum distances
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Update the index of the next cluster center (set cur_id)
        # YOUR CODE HERE
        raise NotImplementedError()

    # Compute cluster size -- this equals the largest 
    # distance from a cluster center to any point that 
    # belongs to the cluster
    #
    # HINT: You already have this information, you just
    #       need to look it up
    # YOUR CODE HERE
    raise NotImplementedError()

    return X[center_indices], center_indices, labels, clust_size


In [ ]:
X = np.eye(4)
centers, center_indices, labels, clust_size = my_k_centers(X, n_clusters=4, seed=0)

assert(centers.sum() == 4)
assert((centers.sum(axis=1) == np.ones((4,))).all())

assert(center_indices.shape == (4,))
assert(center_indices.min() == 0)
assert(center_indices.max() == 3)

assert(labels.shape == (4,))
assert(labels.min() == 0)
assert(labels.max() == 3)

assert(clust_size == 0.0)

assert(centers.shape == (4,X.shape[0]))

## Problem 3

Describe a dataset that "breaks" `my_k_centers`. That is, come up with a dataset that has an expected cluster structure, but the labels that `my_k_centers` returns doesn't match that expectation.

Feel free to use a plot to help your description (you can add a new cell if necessary).

YOUR ANSWER HERE

## Using metrics

In the next two problems we will use the class from the `kcenter.py` file included with the assignment.

The class `KCenters` from this file works like other clustering algorithms from scikit-learn, and implements FurthestPoint, with the exception that we can use any metric that we like. We'll set up some data:

In [ ]:
# Sample from a square with corners (-1,-1) and (1,1)
X = check_random_state(20160901).uniform(-1,1,size=(10000,2))

### $k$-centers with $\ell_2$

The following cell computes $k$-centers on $X$ with the $\ell_2$ or Euclidean metric, like you implemented above. 

In [ ]:
def plot_kcenters_with(X_, n_clusters=8, metric='l2', seed=0):
    
    from kcenter import KCenters
    clust_ = KCenters(n_clusters=n_clusters, metric=metric, seed=seed)
    
    df_ = pd.DataFrame(X_, columns=['X0','X1'])
    df_['ypred'] = clust_.fit_predict(X_)

    # Plot the cluster assignments:
    g = sns.FacetGrid(data=df_, hue='ypred', 
                      size=7, palette='Dark2', 
                      subplot_kws=dict(aspect='equal')) 
    g.map(plt.scatter,'X0','X1')

    # Add the cluster centers with 'X' markers:
    g.ax.scatter(x=clust_.cluster_centers_[:,0], 
                 y=clust_.cluster_centers_[:,1], 
                 marker='x', s=200, c='black', linewidths=2)
    
    return clust_

plot_kcenters_with(X, metric='l2')

If you were to use `my_k_centers` on the data, then you should get the same result:

In [ ]:
centers, center_indices, labels, clust_size = my_k_centers(X, n_clusters=8, seed=0)
df = pd.DataFrame(X, columns=['X0','X1'])
df['ypred'] = labels

# Plot the cluster assignments:
g = sns.FacetGrid(data=df, hue='ypred', size=7, palette='Dark2', subplot_kws=dict(aspect='equal')) 
g.map(plt.scatter,'X0','X1')

# Add the cluster centers with 'X' markers:
g.ax.scatter(x=centers[:,0], y=centers[:,1],marker='x',s=200,c='black', linewidths=2)

## Problem 4

Let's try the same with $\ell_1$. Observe the boundaries between clusters. Why are the boundaries made up of lines at multiples of 45 degrees?

In [ ]:
plot_kcenters_with(X, metric='l1')

YOUR ANSWER HERE

## Problem 5

Now let's test with the cosine metric. Why Do we see a radial structure?

In [ ]:
clust = plot_kcenters_with(X, metric='cosine')
clust = plot_kcenters_with(X+np.array([[.5,.5]]), metric='cosine')

YOUR ANSWER HERE